<a href="https://colab.research.google.com/github/amanigem/CT_SLICE_SORTING_SPARSE_NEURAL_NETWORK/blob/master/CTORG2ARRAY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd

/content/drive/My Drive/CT-ORG-GZ


In [ ]:
cd/content/drive/My Drive/CT-ORG-GZ

/content/drive/My Drive/CT-ORG-GZ


In [ ]:
#!pip install SimpleITK

In [ ]:
#import SimpleITK as sitk

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Activation, Dense, Flatten
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
#from tensorflow.keras.applications.imagenet_utils import preprocess_input
from keras.utils import np_utils
import numpy as np
#from glob import glob
import matplotlib.pyplot as plt
import os
import time
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
import nibabel as nib
#from PIL import Image
#import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.ticker import MultipleLocator
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import argparse
import imutils
import cv2
import matplotlib.image as mpimg
from nibabel.testing import data_path
from numpy.random import default_rng 

def histogramEqualization(f, bins=100):
    his, be = np.histogram(f, range=(0,1), bins=bins)
    his = his.astype(float)/sum(his)
    return np.interp(f, be, np.hstack((np.zeros((1)), np.cumsum(his))))

def release_list(a):
  del a


def release_list6(a1,a2,a3,a4,a5,a6):
   del a1
   del a2
   del a3
   del a4
   del a5
   del a6

def release_list3(a1,a2,a3):
   del a1
   del a2
   del a3

def simplearray(w,h,d1,d2):
  #w,h,d1,d2 = 8, 5,2,5;
  Matrix = [[[[0 for x in range(w)] for y in range(h)] for y in range(d1)]for y in range(d2)]
  return Matrix


# Loading of data with name CTORG from drive
PATH = os.getcwd()
# Define data path
data_path = '/content/drive/My Drive/CT-ORG-GZ'
data_path_vol = data_path+'/volume'
data_path_label = data_path+'/labels'
data_dir_list = os.listdir(data_path)
img_data_list=[]
data_dir_vol_list = os.listdir(data_path_vol)
img_data_vol_list=[]
data_dir_label_list = os.listdir(data_path_label)
img_data_label_list=[]
no_of_slices_arr = np.array([])
label_fln_temp = data_path_label+'/labels-'+str(1)+'.nii.gz'
label_img_temp = nib.load(label_fln_temp)
label_data_temp = label_img_temp.get_fdata()
slice_label_temp = label_data_temp[:,:,1]
#x1=np.array([0,0,0,0,0,0,0])

org_class = ['background ','liver', 'bladder', 'lung', 'kidney', 'bone', 'brain']
org_pref_id_doct = np.array([[0,6],[1,0],[2,4],[3,1],[4,5],[5,2],[6,3]])
org_pref_id_doct1 = np.array([[6,0],[0,1],[4,2],[1,3],[5,4],[2,5],[3,6]])
org_perf_name_doct = ['liver','lung','bone','brain','bladder','kidney','background']
lab_id = []
imgid = 0
'''
ns = 0
for img in data_dir_vol_list:
  label_fln = data_path_label+'/labels-'+str(imgid)+'.nii.gz'
  images = sitk.ReadImage(label_fln)
  imgid += 1
  ns = ns+images.GetDepth()
'''

noofpatient = 20
noofimages = 1 
flag = 0
ns = 63503

#RGB HAVING SAME IMAGES IN THREE CHANNELS
vol32O = []#np.zeros((ns,32,32,3))
vol64O = [] 
vol128O = []
vol256O = []
vol224O = []
vol512O = []

#H HISTOGRAM
vol32H = []#np.zeros((ns,32,32,3))
vol64H = [] 
vol128H = []
vol256H = []
vol224H = []
vol512H = []

#F FILTER
vol32F = []#np.zeros((ns,32,32,3))
vol64F = [] 
vol128F = []
vol256F = []
vol224F = []
vol512F = []

#OHF ORIGINAL, HISTOGRAM, FILTER
vol32OHF = []#np.zeros((ns,32,32,3))
vol64OHF = [] 
vol128OHF = []
vol256OHF = []
vol224OHF = []
vol512OHF = []

dim32 = (32,32)
#dim64 = (64,64)
#dim128 = (128,128)
#dim256 = (256,256)
#dim224 = (224,224)
#dim512 = [512,512]

kernel  = np.ones((3,3), np.float32)/9

no_of_slices_arr = np.array([])
label_fln_temp=data_path_label+'/labels-'+str(1)+'.nii.gz'
label_img_temp = nib.load(label_fln_temp)
label_data_temp = label_img_temp.get_fdata()
slice_label_temp = label_data_temp[:,:,1]
#organ_class_arr=np.array(x1)

count1 = -1
for img in data_dir_vol_list:
#for img in 'volume-4.nii.gz': 
  if (imgid > noofpatient ) :
    break
  vol_fln = data_path_vol+'/volume-'+str(imgid)+'.nii.gz'
  vol_img = nib.load(vol_fln)
  vol_data = vol_img.get_fdata()
  #release_list(vol_img)
  
  label_fln = data_path_label+'/labels-'+str(imgid)+'.nii.gz'
  label_img = nib.load(label_fln)
  label_data = label_img.get_fdata()
  #release_list(label_img)

  #data_vol_label = np.concatenate((vol_data,label_data), axis=2)
  numslices_vol = vol_data.shape[2]
  numslices_label = label_data.shape[2]
  numslices = numslices_vol
  no_of_slices_arr = np.concatenate((no_of_slices_arr,np.array([numslices])))
  scanid = 0
  for scanid in range(0,numslices):
    j = scanid*noofimages
    slice_vol_temp = vol_data[:,:,j]
    slice_vol = slice_vol_temp
    slice_vol_temp = cv2.normalize(slice_vol, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    slice_vol32a = cv2.resize(slice_vol_temp, dim32, interpolation = cv2.INTER_AREA)
    #slice_vol64a = cv2.resize(slice_vol_temp, dim64, interpolation = cv2.INTER_AREA)
    #slice_vol128a = cv2.resize(slice_vol_temp, dim128, interpolation = cv2.INTER_AREA)
    #slice_vol256a = cv2.resize(slice_vol_temp, dim256, interpolation = cv2.INTER_AREA)
    #slice_vol224a = cv2.resize(slice_vol_temp, dim224, interpolation = cv2.INTER_AREA)
    #slice_vol512a = (slice_vol_temp)
    slice_vol32b = histogramEqualization(slice_vol32a)
    slice_vol32b = np.float32(slice_vol32b)
    slice_vol32c = cv2.filter2D(slice_vol32a, -1, kernel )
    #slice_vol64b = histogramEqualization(slice_vol64a)
    #slice_vol64b = np.float32(slice_vol64b)
    #slice_vol64c = cv2.filter2D(slice_vol64a, -1, kernel )
    '''
    slice_vol128b = histogramEqualization(slice_vol128a)
    slice_vol128b = np.float32(slice_vol128b)
    slice_vol128c = cv2.filter2D(slice_vol128a, -1, kernel )
    '''
    '''
    slice_vol256b = histogramEqualization(slice_vol256a)
    slice_vol256b = np.float32(slice_vol256b)
    slice_vol256c = cv2.filter2D(slice_vol256a, -1, kernel )
    '''
    #slice_vol224b = histogramEqualization(slice_vol224a)
    #slice_vol224b = np.float32(slice_vol224b)
    #slice_vol224c = cv2.filter2D(slice_vol224a, -1, kernel )
    #slice_vol512b = histogramEqualization(slice_vol512a)
    #slice_vol512b = np.float32(slice_vol512b)
    #slice_vol512c = cv2.filter2D(slice_vol512a, -1, kernel )

    slice_vol32O= cv2.merge((slice_vol32a,slice_vol32a,slice_vol32a))
    slice_vol32H= cv2.merge((slice_vol32b,slice_vol32b,slice_vol32b))
    slice_vol32F= cv2.merge((slice_vol32c,slice_vol32c,slice_vol32c))
    slice_vol32OHF = cv2.merge((slice_vol32a,slice_vol32b,slice_vol32c))
    '''
    slice_vol64O = cv2.merge((slice_vol64a,slice_vol64a,slice_vol64a))
    slice_vol64H = cv2.merge((slice_vol64b,slice_vol64b,slice_vol64b))
    slice_vol64F = cv2.merge((slice_vol64c,slice_vol64c,slice_vol64c))
    slice_vol64OHF = cv2.merge((slice_vol64a,slice_vol64b,slice_vol64c))
    '''
    '''
    slice_vol128O = cv2.merge((slice_vol128a,slice_vol128a,slice_vol128a))
    slice_vol128H = cv2.merge((slice_vol128b,slice_vol128b,slice_vol128b))
    slice_vol128F = cv2.merge((slice_vol128c,slice_vol128c,slice_vol128c))
    slice_vol128OHF = cv2.merge((slice_vol128a,slice_vol128b,slice_vol128c))
    
    slice_vol256O = cv2.merge((slice_vol256a,slice_vol256a,slice_vol256a))
    slice_vol256H = cv2.merge((slice_vol256b,slice_vol256b,slice_vol256b))
    slice_vol256F = cv2.merge((slice_vol256c,slice_vol256c,slice_vol256c))
    slice_vol256OHF = cv2.merge((slice_vol256a,slice_vol256b,slice_vol256c))
    '''
    #slice_vol224O = cv2.merge((slice_vol224a,slice_vol224a,slice_vol224a))
    #slice_vol224H = cv2.merge((slice_vol224b,slice_vol224b,slice_vol224b))
    #slice_vol224F = cv2.merge((slice_vol224c,slice_vol224c,slice_vol224c))
    #slice_vol224OHF = cv2.merge((slice_vol224a,slice_vol224b,slice_vol224c))

    #slice_vol512O = cv2.merge((slice_vol512a,slice_vol512a,slice_vol512a))
    #slice_vol512H = cv2.merge((slice_vol512b,slice_vol512b,slice_vol512b))
    #slice_vol512F = cv2.merge((slice_vol512c,slice_vol512c,slice_vol512c))
    #slice_vol512OHF = cv2.merge((slice_vol512a,slice_vol512b,slice_vol512c))

    #count1 += 1
    #vol32O[count1,:,:,:] = slice_vol32O
    vol32O.append(slice_vol32O)
    #vol64O.append(slice_vol64O)
    #vol128O.append(slice_vol128O)
    #vol256O.append(slice_vol256O)
      #vol224O.append(slice_vol224O)
    #vol512O.append(slice_vol512O)

      #vol32H[count1,:,:,:] = slice_vol32H
    vol32H.append(slice_vol32H)
    #vol64H.append(slice_vol64H)
    #vol128H.append(slice_vol128H)
    #vol256H.append(slice_vol256H)
      #vol224H.append(slice_vol224H)
    #vol512H.append(slice_vol512H)
      
      #vol32F[count1,:,:,:] = slice_vol32F
    vol32F.append(slice_vol32F)
    #vol64F.append(slice_vol64F)
    #vol128F.append(slice_vol128F)
    #vol256F.append(slice_vol256F)
      #vol224F.append(slice_vol224F)
    #vol512F.append(slice_vol512F)

    #vol32OHF[count1,:,:,:] = slice_vol32OHF
    vol32OHF.append(slice_vol32OHF)
    #vol64OHF.append(slice_vol64OHF)
    #vol128OHF.append(slice_vol128OHF)
    #vol256OHF.append(slice_vol256OHF)
      #vol224OHF.append(slice_vol224OHF)
    #vol512OHF.append(slice_vol512OHF)

      #release_list6(slice_vol32O, slice_vol64O, slice_vol128O, slice_vol256O, slice_vol224O, slice_vol512O)
      #release_list6(slice_vol32H, slice_vol64H, slice_vol128H,slice_vol256H, slice_vol224H,slice_vol512H )
      #release_list6(slice_vol32F, slice_vol64F, slice_vol128F, slice_vol256F,slice_vol224F,slice_vol512F )
      #release_list6(slice_vol32OHF, slice_vol64OHF,slice_vol128OHF,slice_vol256OHF,slice_vol224OHF,slice_vol512OHF)


    slice_label = label_data[:,:,j]
    if(flag):
      
      fig, (ax1, ax2, ax3,ax4,ax5) = plt.subplots(5,1)
      #ax1.set_title('volume image')
      ax1.imshow(slice_vol64O,cmap='jet')
      ax1.axis('off')
      #ax2.set_title('volume image')
      ax2.imshow(slice_vol64H,cmap='jet')
      ax2.axis('off')
      #ax3.set_title('volume image')
      ax3.imshow(slice_vol64F,cmap='jet')
      ax3.axis('off')
      #ax4.set_title('volume image')
      ax4.imshow(slice_vol64OHF,cmap='jet')
      ax4.axis('off')
      #ax5.set_title('label image')
      ax5.imshow(slice_label,cmap='Greys')
      #ax5.axis('off')
      #ax6.set_title('hist_label')
      n_labels,bins_labels,patches_lables = ax5.hist(slice_label.ravel(), bins=7,  range=(0.0, 6), fc='k', ec='k')
      '''
      plt.imshow(slice_vol512O,cmap='jet')
      plt.axis('off')
      plt.show()
      plt.imsave('o.png',slice_vol512O)
      plt.imshow(slice_vol512H,cmap='jet')
      plt.axis('off')
      plt.show()
      plt.imsave('H.png',slice_vol512H)
      plt.imshow(slice_vol512F,cmap='jet')
      plt.axis('off')
      plt.show()
      plt.imsave('F.png',slice_vol512F)
      plt.imshow(slice_vol512OHF,cmap='jet')
      plt.axis('off')
      plt.show()
      plt.imsave('OHF.png',slice_vol512OHF)
      plt.imshow(slice_label,cmap='jet')
      plt.axis('off')
      plt.show()
      plt.imsave('Lable.png',slice_label)
      n_labels,bins_labels,patches_lables = plt.hist(slice_label.ravel(), bins=7,  range=(0.0, 6), fc='k', ec='k')
      plt.show()
      '''
      #plt.savefig('Histogram.png',slice_label.ravel())
    else:
      n_labels,bins_labels,patches_lables = plt.hist(slice_label.ravel(), bins=7,  range=(0.0, 6), fc='k', ec='k')
      plt.close()
    n_labels[n_labels>1]=1
    a1 = n_labels.astype(int)
    b1 = []
    for i in range ( 0 , a1.shape[0]):
      b1.append(a1[i]) 
    a =  b1
    no1 = sum(a)
    p = []
    for i in range (0 , no1):
      p1 = a.index(1)
      p.append(p1) 
      a[p1] = 0
    org_pref_id_doct[p,:]
    b = org_pref_id_doct[p,:]
    c = b[:,1]
    d = np.sort(c)
    B = np.reshape(d, (-1, 1))
    lab_id.append(B[0][0])
    '''
    for i in range (0, d.shape[0]):
      F = org_perf_name_doct[B[i][0]]
    organ_class_arr=np.vstack((organ_class_arr,n_labels))
    '''
      #plt.show()
      #plt.savefig(str(j))
      #plt.close('all') #
    #else:
      #continue
    '''
    if (imgid == 18):
      np.save('vnp18', np.stack(vol32O, axis = 0))
      np.save('Label18',lab_id)
    '''
  else:
    print(imgid)
    imgid=imgid+1

vol32np1 = vol32O#np.stack(vol32O, axis = 0)
#vol64np1 = np.stack(vol64O, axis = 0)
#vol128np1 = np.stack(vol128O, axis = 0)
#vol256np1 = np.stack(vol256O, axis = 0)
#vol224np1 = np.stack(vol224O, axis = 0)
#vol512np1 = np.stack(vol512O, axis = 0)
vol32np2 = vol32H#np.stack(vol32H, axis = 0)
#vol64np2 = np.stack(vol64H, axis = 0)
#vol128np2 = np.stack(vol128H, axis = 0)
#vol256np2 = np.stack(vol256H, axis = 0)
#vol224np2 = np.stack(vol224H, axis = 0)
#vol512np2 = np.stack(vol512H, axis = 0)
vol32np3 = vol32F#np.stack(vol32F, axis = 0)
#vol64np3 = np.stack(vol64F, axis = 0)
#vol128np3 = np.stack(vol128F, axis = 0)
#vol256np3 = np.stack(vol256F, axis = 0)
#vol224np3 = np.stack(vol224F, axis = 0)
#vol512np3 = np.stack(vol512F, axis = 0)
vol32np4 = vol32OHF#np.stack(vol32OHF, axis = 0)
#vol64np4 = np.stack(vol64OHF, axis = 0)
#vol128np4 = np.stack(vol128OHF, axis = 0)
#vol256np4 = np.stack(vol256OHF, axis = 0)
#vol224np4 = np.stack(vol224OHF, axis = 0)
#vol512np4 = np.stack(vol512OHF, axis = 0)

np.save('vol32np1', vol32np1)
#np.save('vol64np1', vol64np1)
#np.save('vol128np1', vol128np1)
#np.save('vol256np1', vol256np1)
#np.save('vol224np1', vol224np1)
#np.save('vol512np1', vol512np1)

np.save('vol32np2', vol32np2)
#np.save('vol64np2', vol64np2)
#np.save('vol128np2', vol128np2)
#np.save('vol256np2', vol256np2)
#np.save('vol224np2', vol224np2)
#np.save('vol512np2', vol512np2)

np.save('vol32np3', vol32np3)
#np.save('vol64np3', vol64np3)
#np.save('vol128np3', vol128np3)
#np.save('vol256np3', vol256np3)
#np.save('vol224np3', vol224np3)
#np.save('vol512np3', vol512np3)

np.save('vol32np4', vol32np4)
#np.save('vol64np4', vol64np4)
#np.save('vol128np4', vol128np4)
#np.save('vol256np4', vol256np4)
#np.save('vol224np4', vol224np4)
#np.save('vol512np4', vol512np4)

np.save('no_of_slices_arr_fln',no_of_slices_arr ) 
#np.save('organ_class_arr_fln',organ_class_arr ) 
train_test_labels = []
org_pref_id_doct2 = np.transpose(org_pref_id_doct1)
org_pref_id_doct1_firstcl = org_pref_id_doct2[0,:]
for i in range ( 0 , len(lab_id)):
  temp2 = np.where(org_pref_id_doct1_firstcl == lab_id[i])
  temp3 = temp2[0]
  temp4 = temp3[0]
  temp4 = org_pref_id_doct1[temp4][1]  
  train_test_labels.append(temp4)
print(train_test_labels)
np.save('train_test_labels_fln',train_test_labels)

print(len(lab_id))
print(len(train_test_labels))


Using TensorFlow backend.


0
1
